# Using sklearn SVM to classify images

In [ ]:
# first, write a function to convert an image to a numpy array

# we want a set of features where the first column is a numpy array representing the image
# and the second column is the class {fake, real}

In [2]:
import numpy as np
from os import walk
from PIL import Image

In [3]:
data_dir = 'data'
real_dir = f'{data_dir}/all_real'
fake_dir = f'{data_dir}/all_fake'

In [4]:
# define function to process an image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image
        returns an Numpy array
    '''
    # resize image so shortest side is 256, maintaining aspect ratio
    resize_num = 224
    w, h = image.size
    (new_w, new_h) = (1 + (resize_num * w // h), resize_num) if w > h else (resize_num, 1 + (resize_num * h // w))
    resized = image.resize((new_w, new_h))
    
    # crop out center 224x224 pixels
    resized_w, resized_h = resized.size
    w_margin = (resized_w - 224) // 2
    h_margin = (resized_h - 224) // 2
    (left, upper, right, lower) = (w_margin, h_margin, resized_w - w_margin, resized_h - h_margin)
    cropped = resized.crop((left, upper, right, lower))
    
    # convert values from 0-225 to decimals 0-1
    np_image = np.array(cropped)
    np_converted = np_image / 255.0
    
    # normalize via instructions above (hint: look below at provided code in imshow)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    normalized = (np_converted - mean) / std
    
    # re-order color channels
    reordered = np.transpose(normalized, (2, 0, 1))

    return reordered

In [5]:
# test out the process image function on a single image of a real person

bill_murray_image = f'{data_dir}/real_murray.jpg'
processed_img = process_image(Image.open(bill_murray_image))
print(processed_img.shape)
flattened = processed_img.flatten()
print(flattened.shape)

(3, 225, 224)
(151200,)


In [6]:
# build an array of arrays to contain X data
# [[], [], ... []]
# build an array of classes (real, fake) for Y data

# run train, test, split twice, making sure that it's randomized, to get train, val, and test

fake_image_paths = []
for (dirpath, dirnames, filenames) in walk(fake_dir):
    fake_image_paths.extend([f'{dirpath}/{fn}' for fn in filenames])
# print(fake_image_paths[0:10])

real_image_paths = []
for (dirpath, dirnames, filenames) in walk(real_dir):
    real_image_paths.extend([f'{dirpath}/{fn}' for fn in filenames])
# print(real_image_paths[0:10])

all_images = []
# X = []
# y = []
print('processing fake images')
for fake in fake_image_paths:
    all_images.append([*process_image(Image.open(fake)).flatten(), 0])
#     y.append(0)
    break

print('processing real images')
for real in real_image_paths:
    all_images.append([*process_image(Image.open(fake)).flatten(), 1])
#     y.append(1)
    break

# X = np.array(X)
# y = np.array(y)
# print(X[0],X[-1])
# print(y[0],y[-1])
all_images = np.array(all_images)
print(all_images[0])
print(all_images[-1])

# hmm... this is basically 151,200 features per row for less than 600 samples/rows
# that doesn't seem good, but we'll try it anyway

processing fake images
processing real images
[-1.94665639 -1.9809059  -1.92953164 ... -1.17699346 -1.12470588
  0.        ]
[-1.94665639 -1.9809059  -1.92953164 ... -1.17699346 -1.12470588
  1.        ]


In [8]:
from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split

# X_train, X__, y_train, y__ = train_test_split(X, y, test_size=0.6, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X__, y__, test_size=0.5, random_state=42)
# print(X_train.shape, X_val.shape, X_test.shape)

# shuffle, then just manually take out slices for train, val and test
# np.random.shuffle(all_images)
# print(all_images[0])
# print(all_images[-1])


# train is first 400 images
train = all_images[0:400, :]
X_train, y_train = train[:, 0:-1], train[:, -1]
print(X_train.shape, y_train.shape)

# val is 401-500
val = all_images[400:500, :]
X_val, y_val = val[:, 0:-1], val[:, -1]
print(X_val.shape, y_val.shape)

# test is 501-567
test = all_images[500:567, :]
X_test, y_test = test[:, 0:-1], test[:, -1]
print(X_test.shape, y_test.shape)

(2, 151201)
(2, 151200) (2,)
[0. 1.]
